# TriKirby Index

### The TriKirby Index is a methodology inspired by the Kirby Index that quantifies the command and effectiveness of each UCSD Pitcher's pitches ahead of the 2026 season.

## Methodology: TriKirby Index (Percentile-Based Command Metric)

### Overview
The **TriKirby Index** is a pitch-level command metric designed to quantify a pitcher’s ability to consistently repeat release direction and release location. The metric extends the original Kirby Index framework by operating at the **pitch-type level**, normalizing performance **relative to NCAA Division I pitchers**, and producing an interpretable **percentile-based score**.

---

## 1. Pitch-Level Data Construction
All calculations begin from pitch-level TrackMan data. Each pitch includes measurements of:

- Vertical Release Angle (VRA)
- Horizontal Release Angle (HRA)
- Vertical Release Location ($vRel$)
- Horizontal Release Location ($hRel$)

Pitch-level data is retained until aggregation to preserve within-pitch variability.

---

## 2. Pitcher–Pitch Type Aggregation
For each pitcher $p$ and pitch type $t$, release consistency is summarized using the **standard deviation** of each release component:

$$
\sigma_{\text{VRA},p,t}, \quad
\sigma_{\text{HRA},p,t}, \quad
\sigma_{\text{vRel},p,t}, \quad
\sigma_{\text{hRel},p,t}
$$

Lower standard deviation indicates greater command consistency.

---

## 3. Percentile-Based Release Consistency Metrics

For each pitcher \( p \) and pitch type \( t \), release consistency is quantified using the
within-pitch-type **standard deviation** of four release components:

- Vertical Release Angle (VRA)
- Horizontal Release Angle (HRA)
- Vertical Release Location (vRel)
- Horizontal Release Location (hRel)

Lower variability indicates greater repeatability and improved pitch command.

To ensure comparability across NCAA Division I pitchers **within each pitch type**, each
standard deviation is converted into a percentile-based consistency score:

$$
\text{SD}_{p,t}^{\text{pct}} = 1 - \operatorname{rank}_{\text{pct}}\left(\text{SD}_{p,t}\right)
$$

This transformation ensures that:

- Scores lie in the interval \( [0,1] \)
- Higher values correspond to **better command**
- Each pitch type is normalized independently
- Metrics are robust to scale differences across pitch types

The resulting percentile-based release consistency metrics are:

$$
\text{sd\_vra\_pct}, \quad
\text{sd\_hra\_pct}, \quad
\text{sd\_vrel\_pct}, \quad
\text{sd\_hrel\_pct}
$$

---

## 4. Linear Component Weights

Each release component contributes differently to overall pitch command.
To reflect this, a set of **fixed linear weights** \( \beta_i \) is applied to each percentile-based
metric.

The weights are normalized such that:

$$
\sum_{i=1}^{4} \beta_i = 1
$$

These weights encode the relative importance of directional consistency
(release angles) versus spatial consistency (release location).

---

## 5. TriKirby Index (Final Score)

The **TriKirby Index** for pitcher \( p \) and pitch type \( t \) is computed as a weighted linear
combination of percentile-based release consistency metrics:

$$
\text{TriKirby}_{p,t}
= \beta_1 \cdot \text{sd\_vra\_pct}_{p,t}
+ \beta_2 \cdot \text{sd\_hra\_pct}_{p,t}
+ \beta_3 \cdot \text{sd\_vrel\_pct}_{p,t}
+ \beta_4 \cdot \text{sd\_hrel\_pct}_{p,t}
$$

---

## 6. Interpretation

- **Higher TriKirby values indicate better pitch command**
- Reflects tighter release direction and release point consistency
- Fully comparable **within pitch type** across NCAA Division I pitchers
- Designed for pitch-level evaluation rather than cross-pitch comparisons
- NCAA D1 Pitching Kirby Index Average Score across all pitch types is 0.5. Any values below 0.5 indicates a below average command on that pitch. Any values above 0.5 is above average command for that pitch across NCAA D1 baseball.


## 1. Data Preparation & Filtering

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

DATA_DIR = Path("Trackman CSVs")   

In [ ]:
import pandas as pd
import numpy as np

# -------------------------------
# Load the two datasets
# -------------------------------
ncaa_df = pd.read_csv("all_games.csv")
ucsd_df = pd.read_csv("ucsd_fall_pitching_data.csv")  # <-- match your exact filename on the left

# -------------------------------
# Backward-compatible aliases
# (so the rest of the notebook works)
# -------------------------------
all_df = ncaa_df.copy()      # NCAA-wide (used for baselines / percentiles / betas)
ucsd_all = ucsd_df.copy()    # UCSD-only (used for your tables / rankings)

print("Loaded NCAA rows:", len(all_df), "| UCSD rows:", len(ucsd_all))
display(all_df.head(3))
display(ucsd_all.head(3))

/var/folders/zh/lt350y8j697d9gr_ryq4y0gm0000gn/T/ipykernel_29549/756047426.py:7: DtypeWarning: Columns (91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  ncaa_df = pd.read_csv("all_games.csv")


## 2. Pitch-Type–Specific Release Variability Metrics

In [ ]:
df = all_df.copy()

def pick_col(candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

PITCHER_COL = pick_col(["Pitcher", "PitcherName", "PitcherNameFull"])
TEAM_COL    = pick_col(["PitcherTeam", "Team", "Pitcher Team", "PitcherTeamAbbrev"])
PITCHTYPE_COL = pick_col(["TaggedPitchType", "PitchType", "AutoPitchType"])

HRA_COL = pick_col(["HorzRelAngle"])
VRA_COL = pick_col(["VertRelAngle"])

# Horizontal + vertical release POINTS (usually X and Z)
RELX_COL = pick_col(["RelSide"])
RELZ_COL = pick_col(["RelHeight"])

print("Pitcher:", PITCHER_COL)
print("Team:", TEAM_COL)
print("PitchType:", PITCHTYPE_COL)
print("HRA:", HRA_COL)
print("VRA:", VRA_COL)
print("RelX:", RELX_COL)
print("RelZ:", RELZ_COL)

req = [PITCHER_COL, TEAM_COL, PITCHTYPE_COL, HRA_COL, VRA_COL, RELX_COL, RELZ_COL]
missing = [r for r in req if r is None]
if missing:
    raise ValueError("Missing required columns. Fix candidates list above. Missing: " + str(missing))

# Keep only what we need + clean
df = df.dropna(subset=[PITCHER_COL, TEAM_COL, PITCHTYPE_COL, HRA_COL, VRA_COL, RELX_COL, RELZ_COL]).copy()
df[PITCHTYPE_COL] = df[PITCHTYPE_COL].astype(str).str.strip()
df[PITCHER_COL]   = df[PITCHER_COL].astype(str).str.strip()
df[TEAM_COL]      = df[TEAM_COL].astype(str).str.strip()

df.head()

# UC San Diego Pitching

In [4]:
# See the most common team codes (so you can confirm UCSD’s exact label)
df[TEAM_COL].value_counts().head(30)

PitcherTeam
CAL_BEA    7997
LOU_CAR    7779
ARI_SUN    7576
OLE_REB    7558
UCLA       7488
TCU_HFG    7433
FLO_PAN    7422
VAN_COM    7418
JMU_DUK    7318
MIA_HUR    7314
VIR_TEC    7303
SOU_TRO    7257
CAL_MUS    7212
LSU_TIG    7145
NEB        7129
ELO_PHO    7075
ARI_WIL    7054
DUK_BLU    7027
MIC_WOL    7016
RUT_SCA    7015
MT         6991
WES_MOU    6987
STM_GAE    6971
TEN_VOL    6968
JAC_GAM    6967
SOU_GAM    6966
LOY_LIO    6915
CHA_FOR    6912
KAN_JAY    6884
ARK_RAZ    6877
Name: count, dtype: int64

In [5]:
UCSD_CODE = "CSD_TRI"   # <-- based on what you saw in your output

ucsd_pitchers = sorted(df.loc[df[TEAM_COL] == UCSD_CODE, PITCHER_COL].unique())
print("UCSD pitchers found:", len(ucsd_pitchers))
ucsd_pitchers[:30]


UCSD pitchers found: 19


['Cazares, Julian',
 'Custer, Julian',
 'Dalquist, Matthew',
 'Davidson, Garrett',
 'Ernisse, Zach',
 'Gregson, Niccolas',
 'Hasegawa, Sam',
 'Huy, Nathan',
 'King, Devon',
 'Marchetti, Landon',
 'Murdock, Steele',
 'Nickerson, Trevor',
 'Patterson, Garrett',
 'Pelzman, Harry',
 'Remmers, Ethan',
 'Ries, Nathan',
 'Seid, Spencer',
 'Villar, Jake',
 'Weber, Chapman']

# NCAA-Wide Spread Table

In [6]:
MIN_PITCHES_PER_TYPE = 0  # adjust if you want (prevents noisy tiny samples)

g = df.groupby([PITCHER_COL, PITCHTYPE_COL])

spread = g.agg(
    n_pitches=(PITCHTYPE_COL, "size"),
    sd_hra=(HRA_COL, "std"),
    sd_vra=(VRA_COL, "std"),
    sd_relx=(RELX_COL, "std"),
    sd_relz=(RELZ_COL, "std"),
).reset_index()

spread = spread[spread["n_pitches"] >= MIN_PITCHES_PER_TYPE].dropna()
spread.head()

,Pitcher,TaggedPitchType,n_pitches,sd_hra,sd_vra,sd_relx,sd_relz
0,"Abad, Adrian",Sinker,31,0.829183,1.192912,0.196204,0.118152
1,"Abad, Adrian",Slider,17,1.254848,1.380489,0.219038,0.119576
2,"Abbadessa, Jude",ChangeUp,7,1.005730,1.908953,0.290681,0.136591
3,"Abbadessa, Jude",Fastball,174,0.837360,0.954544,0.216094,0.114650
4,"Abbadessa, Jude",Sinker,20,0.751234,1.019078,0.300658,0.126558


In [7]:
# --- UCSD-only filter (do this BEFORE building spread) ---
TEAM_COL = "PitcherTeam"  # change if your df uses a different team column
UCSD_TEAM_NAMES = ["CSD_TRI", "UCSD", "SAN_DIEGO", "UC San Diego"]

print("Unique teams (sample):", df[TEAM_COL].dropna().astype(str).unique()[:20])

df = df[df[TEAM_COL].isin(UCSD_TEAM_NAMES)].copy()
print("Rows after UCSD filter:", len(df))

Unique teams (sample): ['SOU_LIO' 'LIN_UNI' 'OKL_COW' 'CLE_TIG' 'COL_CHA' 'VCU_RAM' 'ARK_RAZ'
 'WAS_COU' 'BAY_BEA' 'YSU_PEN' 'LON_DIR' 'NOR_CAT' 'GEO_BUL' 'QUI_BOB'
 'DIX_STE' 'NCB' 'CAL_BEA' 'NEV_WOL' 'ECU_PIR' 'GEO_PAT']
Rows after UCSD filter: 4459


# NCAA Percentile-Based Command Scores

In [8]:
# Percentile-based command (higher = better)
for col in ["sd_hra", "sd_vra", "sd_relx", "sd_relz"]:
    spread[f"{col}_pct"] = 1 - spread.groupby(PITCHTYPE_COL)[col].rank(pct=True)

# Regression Model for Beta Weights

In [9]:
# Select Features + Target (Plate Location)

# Features (predictors)
FEATURES = [
    "VertRelAngle",   # VRA
    "HorzRelAngle",   # HRA
    "RelHeight",      # vRel
    "RelSide"         # hRel
]

# Targets (plate location)
TARGETS = [
    "PlateLocHeight",  # Z location
    "PlateLocSide"     # X location
]

model_df = df.dropna(subset=FEATURES + TARGETS)

X = model_df[FEATURES]
y = model_df[TARGETS]


In [10]:
# Test/Train Split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [11]:
# Linear Regression (Kirby-style)

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

y_pred = lin_reg.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))


MSE: 0.5178465982081192
R²: 0.41230259350869847


In [12]:
# Extract Beta Weights

beta_df = pd.DataFrame(
    lin_reg.coef_,
    columns=FEATURES,
    index=["PlateSide", "PlateHeight"]
)

beta_df

,VertRelAngle,HorzRelAngle,RelHeight,RelSide
PlateSide,0.365545,-0.046137,0.602566,-0.023966
PlateHeight,-0.046994,0.607446,0.542632,0.727966


In [13]:
beta_weights = beta_df.abs().mean(axis=0)
beta_weights = beta_weights / beta_weights.sum()  # normalize

beta_weights

VertRelAngle    0.139218
HorzRelAngle    0.220563
RelHeight       0.386467
RelSide         0.253752
dtype: float64

# Build Pitch Command Spread (Raw SDs)

In [14]:
import numpy as np

# --- 1) Identify the correct release point columns in YOUR CSV ---
# Trackman commonly uses RelHeight (vertical release) and RelSide (horizontal release)
CAND_VREL = ["RelHeight", "ReleaseHeight", "release_pos_z", "RelZ", "vRel"]
CAND_HREL = ["RelSide", "ReleaseSide", "release_pos_x", "RelX", "hRel"]

vrel_col = next((c for c in CAND_VREL if c in df.columns), None)
hrel_col = next((c for c in CAND_HREL if c in df.columns), None)

print("Using vRel column:", vrel_col)
print("Using hRel column:", hrel_col)

if vrel_col is None or hrel_col is None:
    raise ValueError(
        f"Could not find release point columns. "
        f"Columns in df include: {list(df.columns)[:40]} ..."
    )

# --- 2) Compute SDs per (Pitcher, PitchType) ---
PITCHER_COL = "Pitcher"
PITCHTYPE_COL = "TaggedPitchType"

spread = (
    df.groupby([PITCHER_COL, PITCHTYPE_COL])
      .agg(
          sd_vra=("VertRelAngle", lambda s: s.std(ddof=1)),
          sd_hra=("HorzRelAngle", lambda s: s.std(ddof=1)),
          sd_vrel=(vrel_col, lambda s: s.std(ddof=1)),
          sd_hrel=(hrel_col, lambda s: s.std(ddof=1)),
          n=("VertRelAngle", "size")
      )
      .reset_index()
)

# (Optional but recommended) drop tiny sample sizes to reduce noise
MIN_PITCHES_PER_TYPE = 0
spread = spread[spread["n"] >= MIN_PITCHES_PER_TYPE].copy()

# --- 3) Z-score each SD metric within pitch type (NCAA-wide benchmark) ---
def z_by_pitchtype(series):
    mu = series.mean()
    sd = series.std(ddof=0)
    return (series - mu) / sd if sd != 0 else np.nan

spread["z_sd_vra"]  = spread.groupby(PITCHTYPE_COL)["sd_vra"].transform(z_by_pitchtype)
spread["z_sd_hra"]  = spread.groupby(PITCHTYPE_COL)["sd_hra"].transform(z_by_pitchtype)
spread["z_sd_vrel"] = spread.groupby(PITCHTYPE_COL)["sd_vrel"].transform(z_by_pitchtype)
spread["z_sd_hrel"] = spread.groupby(PITCHTYPE_COL)["sd_hrel"].transform(z_by_pitchtype)

spread.head()

Using vRel column: RelHeight
Using hRel column: RelSide


,Pitcher,TaggedPitchType,sd_vra,sd_hra,sd_vrel,sd_hrel,n,z_sd_vra,z_sd_hra,z_sd_vrel,z_sd_hrel
0,"Cazares, Julian",ChangeUp,0.262306,0.723429,0.146535,0.175095,3,-2.078778,-0.441253,1.374577,-0.278438
1,"Cazares, Julian",Cutter,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
2,"Cazares, Julian",Fastball,0.877410,0.721784,0.153933,0.146281,38,-1.117055,-0.743044,0.675611,-0.536474
3,"Cazares, Julian",FourSeamFastBall,0.754681,2.076283,0.045497,0.086328,3,-1.030274,2.022747,-0.737004,-0.608437
4,"Cazares, Julian",Slider,1.129368,1.095653,0.118094,0.143436,29,-0.401259,-0.111984,0.227484,-0.391480


## NCAA-Normalized Release Consistency Percentiles

In [15]:
# ============================================================
# Convert release SDs to percentiles (lower SD = better)
# Done per pitch type (NCAA-normalized)
# Creates: sd_vra_pct, sd_hra_pct, sd_vrel_pct, sd_hrel_pct
# ============================================================

PCT_COLS = {
    "sd_vra":  "sd_vra_pct",
    "sd_hra":  "sd_hra_pct",
    "sd_vrel": "sd_vrel_pct",
    "sd_hrel": "sd_hrel_pct",
}

missing_raw = [c for c in PCT_COLS.keys() if c not in spread.columns]
if missing_raw:
    raise ValueError(f"Missing raw SD columns needed for percentiles: {missing_raw}")

for raw_col, pct_col in PCT_COLS.items():
    # rank within each pitch type; invert so lower SD = higher percentile (better)
    spread[pct_col] = 1 - spread.groupby(PITCHTYPE_COL)[raw_col].rank(pct=True)

# round for display (hundredths)
for c in PCT_COLS.values():
    spread[c] = spread[c].round(2)

display(spread[list(PCT_COLS.values())].describe())

,sd_vra_pct,sd_hra_pct,sd_vrel_pct,sd_hrel_pct
count,84.000000,84.000000,84.000000,84.000000
mean,0.434524,0.434524,0.434524,0.434524
std,0.296507,0.296507,0.296507,0.296507
min,0.000000,0.000000,0.000000,0.000000
25%,0.167500,0.167500,0.167500,0.167500
50%,0.440000,0.440000,0.440000,0.440000
75%,0.672500,0.672500,0.672500,0.672500
max,0.950000,0.950000,0.950000,0.950000


# TriKirby Equation - Composite Pitch Command Metric

In [16]:
# TriKirby Equation (percentile-based, higher = better)

betas = {
    "sd_vra_pct": beta_weights["VertRelAngle"],
    "sd_hra_pct": beta_weights["HorzRelAngle"],
    "sd_vrel_pct": beta_weights["RelHeight"],
    "sd_hrel_pct": beta_weights["RelSide"],
}

spread["TriKirby"] = (
    betas["sd_vra_pct"]  * spread["sd_vra_pct"]  +
    betas["sd_hra_pct"]  * spread["sd_hra_pct"]  +
    betas["sd_vrel_pct"] * spread["sd_vrel_pct"] +
    betas["sd_hrel_pct"] * spread["sd_hrel_pct"]
).round(3)

display(
    spread[
        ["Pitcher", "TaggedPitchType", "TriKirby",
         "sd_vra_pct", "sd_hra_pct", "sd_vrel_pct", "sd_hrel_pct"]
    ].head()
)

,Pitcher,TaggedPitchType,TriKirby,sd_vra_pct,sd_hra_pct,sd_vrel_pct,sd_hrel_pct
0,"Cazares, Julian",ChangeUp,0.448,0.92,0.69,0.08,0.54
1,"Cazares, Julian",Cutter,NaN,NaN,NaN,NaN,NaN
2,"Cazares, Julian",Fastball,0.520,0.84,0.79,0.21,0.58
3,"Cazares, Julian",FourSeamFastBall,0.501,0.83,0.00,0.67,0.50
4,"Cazares, Julian",Slider,0.475,0.58,0.47,0.37,0.58


# UCSD TriKirby Index Score Across all Pitch Types (NCAA D1 AVG TriKirby Index Score = 0.5)

In [17]:
# ================================
# UCSD TriKirby by Pitch Type
# Percentile-based (0–1), higher = better
# ================================

ucsd_spread = spread.copy()

pitch_types = sorted(ucsd_spread[PITCHTYPE_COL].dropna().unique())
print("UCSD pitch types found:", pitch_types)

for pt in pitch_types:
    df_pt = (
        ucsd_spread[ucsd_spread[PITCHTYPE_COL] == pt]
        .sort_values("TriKirby", ascending=False)
        .copy()
    )

    display_cols = [
        PITCHER_COL,
        PITCHTYPE_COL,
        "n",
        "TriKirby",
        "sd_vra_pct",
        "sd_hra_pct",
        "sd_vrel_pct",
        "sd_hrel_pct",
    ]
    display_cols = [c for c in display_cols if c in df_pt.columns]

    print(f"\n=== {pt} (UCSD only) — TriKirby Command Percentile ===")
    display(df_pt[display_cols].reset_index(drop=True))

UCSD pitch types found: ['ChangeUp', 'Curveball', 'Cutter', 'Fastball', 'FourSeamFastBall', 'Knuckleball', 'Sinker', 'Slider', 'Splitter', 'Sweeper', 'TwoSeamFastBall']

=== ChangeUp (UCSD only) — TriKirby Command Percentile ===


,Pitcher,TaggedPitchType,n,TriKirby,sd_vra_pct,sd_hra_pct,sd_vrel_pct,sd_hrel_pct
0,"Marchetti, Landon",ChangeUp,10,0.808,0.23,0.85,0.92,0.92
1,"Gregson, Niccolas",ChangeUp,6,0.725,0.08,0.77,0.85,0.85
2,"Dalquist, Matthew",ChangeUp,43,0.657,0.69,0.62,0.69,0.62
3,"King, Devon",ChangeUp,4,0.634,0.54,0.92,0.77,0.23
4,"Nickerson, Trevor",ChangeUp,7,0.498,0.31,0.23,0.54,0.77
5,"Cazares, Julian",ChangeUp,3,0.448,0.92,0.69,0.08,0.54
6,"Villar, Jake",ChangeUp,101,0.436,0.38,0.54,0.23,0.69
7,"Davidson, Garrett",ChangeUp,126,0.398,0.62,0.31,0.38,0.38
8,"Pelzman, Harry",ChangeUp,3,0.378,0.85,0.00,0.62,0.08
9,"Murdock, Steele",ChangeUp,51,0.313,0.46,0.15,0.46,0.15



=== Curveball (UCSD only) — TriKirby Command Percentile ===


,Pitcher,TaggedPitchType,n,TriKirby,sd_vra_pct,sd_hra_pct,sd_vrel_pct,sd_hrel_pct
0,"King, Devon",Curveball,9,0.723,0.78,0.33,0.89,0.78
1,"Gregson, Niccolas",Curveball,58,0.677,0.44,0.78,0.78,0.56
2,"Dalquist, Matthew",Curveball,75,0.655,0.56,0.67,0.67,0.67
3,"Davidson, Garrett",Curveball,13,0.473,0.22,0.00,0.56,0.89
4,"Hasegawa, Sam",Curveball,4,0.369,0.33,0.44,0.44,0.22
5,"Remmers, Ethan",Curveball,3,0.320,0.89,0.89,0.00,0.00
6,"Marchetti, Landon",Curveball,14,0.287,0.67,0.56,0.11,0.11
7,"Ries, Nathan",Curveball,55,0.261,0.11,0.22,0.22,0.44
8,"Villar, Jake",Curveball,16,0.236,0.00,0.11,0.33,0.33
9,"Murdock, Steele",Curveball,1,NaN,NaN,NaN,NaN,NaN



=== Cutter (UCSD only) — TriKirby Command Percentile ===


...

,Pitcher,TaggedPitchType,n,TriKirby,sd_vra_pct,sd_hra_pct,sd_vrel_pct,sd_hrel_pct
0,"Seid, Spencer",Splitter,18,0.0,0.0,0.0,0.0,0.0



=== Sweeper (UCSD only) — TriKirby Command Percentile ===


,Pitcher,TaggedPitchType,n,TriKirby,sd_vra_pct,sd_hra_pct,sd_vrel_pct,sd_hrel_pct
0,"Seid, Spencer",Sweeper,5,0.32,0.0,0.0,0.5,0.5
1,"Villar, Jake",Sweeper,24,0.18,0.5,0.5,0.0,0.0



=== TwoSeamFastBall (UCSD only) — TriKirby Command Percentile ===


,Pitcher,TaggedPitchType,n,TriKirby,sd_vra_pct,sd_hra_pct,sd_vrel_pct,sd_hrel_pct
0,"Cazares, Julian",TwoSeamFastBall,2,0.307,0.5,0.5,0.0,0.5
1,"King, Devon",TwoSeamFastBall,2,0.193,0.0,0.0,0.5,0.0
